In [1]:
# 加载需要的库
import pandas as pd

In [2]:
#录入aoi数据
aoi=pd.DataFrame(columns=['id','aoi']) 
for data in range(20170306,20170320): 
    aoi_daily=pd.read_csv(r'C:/Users/asus/Downloads/lookalike/'+str(data)+r'/aoi/part-00000',header=None,names=['id','aoi'],sep='\t')
    aoi=aoi.append(aoi_daily)

In [4]:
#拆分aoi变量
aoi_split=aoi['aoi'].str.split(' ',expand=True)
aoi_split.index=aoi.set_index('id').index

In [5]:
#将宽数据转换为长数据
aoi_stack=aoi_split.stack().reset_index('id')

In [6]:
#删除缺失值
aoi_stack.columns=('id','aoi')
aoi_stack=aoi_stack[~aoi_stack['aoi'].isin(['null'])]

In [7]:
#将aoi转换为哑变量
aoi_onehot=pd.get_dummies(aoi_stack)

In [8]:
#合并同一个id下的aoi值
aoi_cnt=aoi_onehot.groupby('id').sum()

In [9]:
#录入dev数据
dev = pd.DataFrame(columns=['id','dev'])
for date in range(20170306,20170319):
    dev_daily=pd.read_csv(r'C:/Users/asus/Downloads/lookalike/'+str(data)+r'/dev/part-00000',header=None,names=['id','dev'],sep='\t')
    dev=dev.append(dev_daily)

In [10]:
#录入设备信息
dev_model=pd.read_csv(r'C:/Users/asus/Downloads/lookalike/model_info.csv')
dev_model

,Index,model_name,brand_En,brand_Ch,model,price_interval,price,type,screen,tag,device,pixel
0,1,SAMSUNG:SM-A7009,samsung,三星,Galaxy A7,2000-3999,2000.0,手机,5.5-5.1英寸,高性价比,"8核芯片,双卡双待",1920x1080
1,2,A7LTECTC:SM-A7009,samsung,三星,Galaxy A7,2000-3999,2000.0,手机,5.5-5.1英寸,高性价比,"8核芯片,双卡双待",1920x1080
2,3,HUAWEI:SM-A7009,samsung,三星,Galaxy A7,2000-3999,2000.0,手机,5.5-5.1英寸,高性价比,"8核芯片,双卡双待",1920x1080
3,4,OPPO:SM-A7009,samsung,三星,Galaxy A7,2000-3999,2000.0,手机,5.5-5.1英寸,高性价比,"8核芯片,双卡双待",1920x1080
4,5,XIAOMI:SM-A7009,samsung,三星,Galaxy A7,2000-3999,2000.0,手机,5.5-5.1英寸,高性价比,"8核芯片,双卡双待",1920x1080
...,...,...,...,...,...,...,...,...,...,...,...,...
12386,98144,SONY:F3216,Sony,索尼,Xperia XA1 Ultra17,4000及以上,4799.0,手机,6.0-5.6英寸,摄影,8核芯片，,1920x1080
12387,98145,NX549J,nubia,努比亚,Z11 miniS,1000-1999,1529.0,手机,5.0-4.6英寸,摄影,8核芯片，陀螺仪，NFC芯片指纹识别，双卡双待,1920x1080
12388,98146,KONKA:R8,KONKA,康佳,R8,500-999,799.0,手机,5.0-4.6英寸,摄影,双卡双待,1280x720
12389,98147,R8,KONKA,康佳,R8,500-999,799.0,手机,5.0-4.6英寸,摄影,双卡双待,1280x720


In [11]:
dev_model.isnull().any()

Index             False
model_name        False
brand_En          False
brand_Ch          False
model             False
price_interval     True
price              True
type               True
screen             True
tag                True
device             True
pixel              True
dtype: bool

In [12]:
#删除设备信息数据的不必要信息
dev_model_drop=dev_model.drop(['Index','brand_Ch','brand_En','model','price_interval','device','pixel','screen','type','price','tag'],axis=1)

In [13]:
#将设备信息数据改为dev
dev_model_drop=dev_model_drop.rename(columns={'model_name':'dev'})
dev_model_drop

,dev
0,SAMSUNG:SM-A7009
1,A7LTECTC:SM-A7009
2,HUAWEI:SM-A7009
3,OPPO:SM-A7009
4,XIAOMI:SM-A7009
...,...
12386,SONY:F3216
12387,NX549J
12388,KONKA:R8
12389,R8


In [14]:
#合并人群信息和设备信息
dev_join=dev.merge(dev_model_drop,on='dev',how='left')
dev_join

,id,dev
0,8568,HUAWEI:CAM-TL00
1,8680,YuLong:Coolpad 8675-HD
2,17360,nubia:NX511J_V3
3,19825,:vivo X6Plus A
4,19825,:vivo X6Plus A
...,...,...
471804,55166,"iPhone8,1"
471805,60263,:vivo X9
471806,61182,:OPPO R7sm
471807,65988,"iPhone9,2"


In [15]:
dev_join.duplicated('id')

0         False
1         False
2         False
3         False
4          True
          ...  
471804     True
471805     True
471806     True
471807     True
471808     True
Length: 471809, dtype: bool

In [16]:
#删除重复的行
dev_new=dev_join.drop_duplicates()
dev_new

,id,dev
0,8568,HUAWEI:CAM-TL00
1,8680,YuLong:Coolpad 8675-HD
2,17360,nubia:NX511J_V3
3,19825,:vivo X6Plus A
5,21089,"iPhone7,2"
...,...,...
36288,55166,"iPhone8,1"
36289,60263,:vivo X9
36290,61182,:OPPO R7sm
36291,65988,"iPhone9,2"


In [17]:
len(dev['id'].unique().tolist())#处理完成

35065

In [18]:
dev_new_2=dev_new.copy()
dev_model_full=dev_model.rename(columns={'model_name':'dev'})

In [19]:
dev_new_2=dev_new_2.merge(dev_model_full,on='dev',how='left')
dev_new_2

,id,dev,Index,brand_En,brand_Ch,model,price_interval,price,type,screen,tag,device,pixel
0,8568,HUAWEI:CAM-TL00,96656.0,HUAWEI,华为,荣耀畅玩5A,500-999,799.0,手机,5.5-5.1英寸,摄影,8核芯片，双卡双待,NaN
1,8680,YuLong:Coolpad 8675-HD,85950.0,Coolpad,酷派,大神F2,500-999,729.0,手机,5.5-5.1英寸,摄影，高性价比,8核芯片，陀螺仪，双卡双待,NaN
2,17360,nubia:NX511J_V3,92857.0,Nubia,努比亚,Z9 mini,NaN,NaN,手机,5.0-4.6英寸,NaN,8核芯片，双卡双待,NaN
3,19825,:vivo X6Plus A,91398.0,vivo,vivo,X6 Plus,2000-3999,3198.0,手机,6.0-5.6英寸,音乐，摄影,8核芯片，陀螺仪，指纹识别，双卡双待,NaN
4,19825,:vivo X6Plus A,96058.0,vivo,vivo,X6 Plus,NaN,NaN,手机,6.0-5.6英寸,NaN,8核芯片，陀螺仪，指纹识别,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36288,55166,"iPhone8,1",97515.0,iPhone,苹果,iPhone 6s,4000及以上,5200.0,手机,5.0-4.6英寸,摄影,陀螺仪，NFC芯片，指纹识别,1334x750
36289,60263,:vivo X9,97382.0,vivo,vivo,X9,2000-3999,2798.0,手机,5.5-5.1英寸,音乐，摄影,8核芯片，陀螺仪，指纹识别，双卡双待,1920x1080
36290,61182,:OPPO R7sm,88177.0,OPPO,OPPO,R7s,2000-3999,2599.0,手机,5.5-5.1英寸,美颜,8核芯片，双卡双待,NaN
36291,65988,"iPhone9,2",97540.0,iPhone,苹果,iPhone 7 Plus,4000及以上,6388.0,手机,5.5-5.1英寸,摄影，高端商务，高性价比,陀螺仪，NFC芯片，指纹识别,1920x1080


In [20]:
dev_clear=dev_new_2.loc[~dev_new_2['id'].isnull()]

In [21]:
dev_clear=dev_clear.drop(['brand_Ch','model','price_interval','device','pixel','screen','type','price'],axis=1)

In [22]:
#将商品名，tag变为哑变量
dummies_brand = pd.get_dummies(dev_clear['brand_En'], prefix= 'brand')
dummies_tag = pd.get_dummies(dev_clear['tag'], prefix= 'tag')
device = pd.concat([dev_clear, dummies_brand,dummies_tag], axis=1)
device.drop(['brand_En', 'tag'], axis=1, inplace=True)

In [23]:
device=device.groupby('id').sum()

In [24]:
#查看设备数据集
device

,Index,brand_ASUS,brand_Acer,brand_Allwinner,brand_Amazon,brand_BBK,brand_BESTSONNY,brand_BOWAY,brand_Bestv,brand_Bifer,...,tag_老人机，摄影,tag_老人机，高性价比,tag_音乐,tag_音乐，摄影,tag_音乐，摄影，高端商务,tag_音乐，美颜,tag_音乐，美颜，摄影,tag_音乐，美颜，高性价比,tag_高性价比,tag_高端商务
id,,,,,,,,,,,,,,,,,,,,,
3,97297.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,90874.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,87426.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,182274.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,188186.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141031,97009.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
141036,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141039,96104.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#创建app数据集
app = pd.DataFrame(columns=['id','app'])
for date in range(20170306,20170319):
    app_daily=pd.read_csv(r'C:/Users/asus/Downloads/lookalike/'+str(data)+'/app/part-00000',header=None,names=['id','app'],sep='\t')
    app=app.append(app_daily)

In [26]:
#拆分app数据集的app数据
app_split=app['app'].str.split(' ',expand=True)
app_split.index=app.set_index('id').index
app_split.head()

,0,1,2,3,4,5,6,7,8,9,...,474,475,476,477,478,479,480,481,482,483
id,,,,,,,,,,,,,,,,,,,,,
5043,10770,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8717,67309,65412,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
20578,38122,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
25040,23392,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
25181,86107,61260,20913,86644,56845,7028,25261,30660,81144,33860,...,None,None,None,None,None,None,None,None,None,None


In [27]:
#将宽数据转换为长数据
app_stack=app_split.stack().reset_index('id')

In [28]:
#删除缺失值
app_stack.columns=('id','app')
app_stack=app_stack.dropna(subset=['app'])
app_stack

,id,app
0,5043,10770
0,8717,67309
1,8717,65412
0,20578,38122
0,25040,23392
...,...,...
16,121481,54828
17,121481,37202
18,121481,71816
0,125551,32768


In [29]:
#读取appid信息
app_tag = pd.read_csv(r'C:/Users/asus/Downloads/lookalike/appid_tag/part-00000',header=None,names=['app','app_tag'],sep='\t')
app_tag

,app,app_tag
0,85878,135
1,59192,130
2,30245,179
3,9990,215
4,35770,137
...,...,...
6611,64671,181
6612,62344,179
6613,36643,204
6614,30576,152 152


In [30]:
#拆分appid列
app_tag_split=app_tag['app_tag'].str.split(' ',expand=True)
app2=pd.merge(app_tag,app_tag_split,how='left',left_index=True,right_index=True)
app2=app2.drop(['app_tag'],axis=1)
app2

,app,0,1,2,3,4,5,6,7,8,9,10,11,12
0,85878,135,None,None,None,None,None,None,None,None,None,None,None,None
1,59192,130,None,None,None,None,None,None,None,None,None,None,None,None
2,30245,179,None,None,None,None,None,None,None,None,None,None,None,None
3,9990,215,None,None,None,None,None,None,None,None,None,None,None,None
4,35770,137,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6611,64671,181,None,None,None,None,None,None,None,None,None,None,None,None
6612,62344,179,None,None,None,None,None,None,None,None,None,None,None,None
6613,36643,204,None,None,None,None,None,None,None,None,None,None,None,None
6614,30576,152,152,None,None,None,None,None,None,None,None,None,None,None


In [31]:
#将宽数据转换为长数据并删除缺失值
app_tag_stack=app2.melt(id_vars=['app'], var_name="Type", value_name="tag")
app_tag_stack=app_tag_stack.dropna(subset=['tag'])
app_tag_stack=app_tag_stack.drop(['Type'],axis=1)
app_tag_stack=app_tag_stack.rename(columns={'tag':'tagId'})

In [32]:
#录入APP信息数据集
tag_name = pd.read_csv(r'C:/Users/asus/Downloads/lookalike/tag_name.csv')
tag_name = tag_name.drop(['name','code','isLeaf'],axis=1)
tag_name.head()

,tagId,parentId
0,115,1.0
1,125,1.0
2,302,1.0
3,312,1.0
4,405,1.0


In [33]:
#更改tagid的数据类型
app_tag_stack["tagId"]=app_tag_stack["tagId"].astype(int)
tag_name.head()

,tagId,parentId
0,115,1.0
1,125,1.0
2,302,1.0
3,312,1.0
4,405,1.0


In [34]:
#合并appid与APP类型数据
app_tag_join = app_tag_stack.merge(tag_name,on='tagId')
app_tag_join

,app,tagId,parentId
0,85878,135,133.0
1,41286,135,133.0
2,27471,135,133.0
3,13524,135,133.0
4,71921,135,133.0
...,...,...,...
8369,1785,282,281.0
8370,32590,282,281.0
8371,35706,282,281.0
8372,43668,692,251.0


In [35]:
#更改APP数据类型
app_stack["app"]=app_stack["app"].astype(int)

In [36]:
#合并app信息
app_tag_merge = app_stack.merge(app_tag_join,on='app')

In [37]:
#更改tagid数据类型
app_tag_merge["tagId"]=app_tag_merge["tagId"].astype(object)
app_tag_merge["app"]=app_tag_merge["app"].astype(object)
app_tag_merge["parentId"]=app_tag_merge["parentId"].astype(object)

In [38]:
#将tagid转换为哑变量
dummies_tag = pd.get_dummies(app_tag_merge['tagId'], prefix= 'tag')
app_tag_merge=pd.concat([app_tag_merge, dummies_tag], axis=1)
app_tag_merge.drop(['tagId','app','parentId'], axis=1, inplace=True)
app_tag_merge

,id,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,...,tag_279,tag_280,tag_282,tag_283,tag_284,tag_691,tag_692,tag_694,tag_695,tag_847
0,5043,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5043,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5043,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21250,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21250,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854573,64069,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3854574,64069,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3854575,64069,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3854576,64069,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
#合并人群App数据
app_cnt=app_tag_merge.groupby('id').sum()

In [41]:
df=pd.concat([app_cnt,device],axis=1,join='outer')
df.head()

,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,tag_138,...,tag_老人机，摄影,tag_老人机，高性价比,tag_音乐,tag_音乐，摄影,tag_音乐，摄影，高端商务,tag_音乐，美颜,tag_音乐，美颜，摄影,tag_音乐，美颜，高性价比,tag_高性价比,tag_高端商务
id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df2=pd.concat([app_cnt,device],axis=1,join='inner')

In [42]:
df=pd.concat([df,aoi_cnt],axis=1,join='outer')
df.head()

,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,tag_138,...,aoi_火车站,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场
id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df2=pd.concat([df2,aoi_cnt],axis=1,join='inner')

In [ ]:
#df=df.reset_index()

In [ ]:
df2=df2.reset_index()

df2

In [ ]:
df.to_csv(r'C:/Users/asus/Downloads/lookalike/df')
df2.to_csv(r'C:/Users/asus/Downloads/lookalike/df2')

In [43]:
#读取种子数据
seed1=pd.read_csv(r'C:/Users/asus/Downloads/lookalike/seed/1_visible/part-00000',header=None,names=['id'],sep='\t')

In [44]:
seed_one=pd.DataFrame(columns=['id','seed1'])
seed_one=seed_one.append(seed1)
seed_one['seed1']='1'
seed_one.head()

D:\Program Files (x86)\A\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,id,seed1
0,109845,1
1,121203,1
2,134044,1
3,109445,1
4,120921,1


In [45]:
seed_one.set_index('id')

,seed1
id,
109845,1
121203,1
134044,1
109445,1
120921,1
...,...
8490,1
16673,1
3347,1


In [ ]:
#df.drop(columns='seed1')

In [68]:
seed1_df=df.merge(seed_one,on='id',how='left')

In [69]:
seed1_df

,id,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,...,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场,seed1
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,4,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53527,141036,26.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53528,141039,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53529,141040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
53530,141047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [70]:
seed1_df['seed1'][seed1_df['seed1'].isnull()]=0
seed1_df.head()

D:\Program Files (x86)\A\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,...,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场,seed1
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,4,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [79]:
seed1_full=seed_one.merge(df,on='id',how='left')

In [80]:
seed1_full

,id,seed1,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,...,aoi_火车站,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场
0,109845,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121203,1,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,134044,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109445,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120921,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,8490,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11360,16673,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11361,3347,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11362,14462,1,0.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
seed1_full.fillna(0)

,id,seed1,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,...,aoi_火车站,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场
0,109845,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,121203,1,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,134044,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109445,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120921,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,8490,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11360,16673,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11361,3347,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11362,14462,1,0.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
import numpy as np
seed1_full=seed1_full.astype(np.float)
seed1_df=seed1_df.astype(np.float)

In [105]:
from sklearn.model_selection import train_test_split
#全部用于训练集
X_train,X_test,y_train,y_test=train_test_split(seed1_df.drop(['seed1','id'],axis=1),seed1_df['seed1'],test_size=0.3,random_state=1)

In [106]:
#数据集
import xgboost as xgb
dtrain=xgb.DMatrix(X_train,y_train)
dtest=xgb.DMatrix(X_test,y_test)

In [122]:
#设置XGBoost模型
param={
    #General Parameters
    'booster':'gbtree',#可选择gbtree和gblinear，默认为gbtree
    'silent':1,#取0时表示打印出运行时信息，取1时表示以缄默方式运行
    
    #Booster Parameters
    'eta':0.01,# 学习率，默认为0.3
    'max_depth':15,#树的最大深度，默认为6
    'min_child_weight':1,
    'gamma':1,
    'subsample':0.7,# 用于训练模型的子样本占整个样本集合的比例
    'colsample_bytree':0.7,# 在建立树时特征采样的比例
    
    # Task Parameters
    'objective':'binary:logistic',
    'eval_metric':'auc',# 评价标准
}

#训练次数
num_round=10
#评估结果
evallist=[(dtest,'eval'),(dtrain,'train')]
#训练
bst1 = xgb.train(param,dtrain,num_round,evallist)

[0]	eval-auc:0.750879	train-auc:0.79282
[1]	eval-auc:0.768867	train-auc:0.814813
[2]	eval-auc:0.777934	train-auc:0.821013
[3]	eval-auc:0.77959	train-auc:0.821468
[4]	eval-auc:0.781171	train-auc:0.822562
[5]	eval-auc:0.784306	train-auc:0.823852
[6]	eval-auc:0.786087	train-auc:0.82657
[7]	eval-auc:0.786338	train-auc:0.827503
[8]	eval-auc:0.787306	train-auc:0.827884
[9]	eval-auc:0.787166	train-auc:0.828457


In [126]:
whole_data=xgb.DMatrix(seed1_full.drop(['seed1','id'],axis=1),seed1_full.seed1)
y_predicted=bst1.predict(whole_data)
#查看预测结果
y_predicted

array([0.4607548 , 0.48528346, 0.46371838, ..., 0.4607548 , 0.48883334,
       0.463154  ], dtype=float32)

In [127]:
#查看预测结果
results=pd.DataFrame(columns=['id','result']) 
results['id']=seed1['id']
results['result']=y_predicted

In [128]:
results

,id,result
0,109845,0.460755
1,121203,0.485283
2,134044,0.463718
3,109445,0.463154
4,120921,0.460755
...,...,...
11359,8490,0.481867
11360,16673,0.463392
11361,3347,0.460755
11362,14462,0.488833


In [133]:
#读取种子数据
seed_two=pd.DataFrame(columns=['id','seed2']) 
seed2=pd.read_csv(r'C:/Users/asus/Downloads/lookalike/seed/2_visible/part-00000',header=None,names=['id'],sep='\t')
seed_two=seed_two.append(seed2)

In [134]:
seed_two['seed2']=1
seed_two

,id,seed2
0,61514,1
1,95783,1
2,97461,1
3,106237,1
4,42617,1
...,...,...
1232,61811,1
1233,18288,1
1234,26462,1
1235,88105,1


In [173]:
seed2_df=df.merge(seed_two,on='id',how='left')
seed2_df

,id,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,...,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场,seed2
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53306,141036,26.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53307,141039,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53308,141040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
53309,141047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [174]:
seed2_df['seed2'][seed2_df['seed2'].isnull()]=0
seed2_df.head()

D:\Program Files (x86)\A\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,tag_137,...,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场,seed2
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,4,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [175]:
from sklearn.model_selection import train_test_split
#全部用于训练集
X_train2,X_test2,y_train2,y_test2=train_test_split(seed2_df.drop(['seed2','id'],axis=1),seed2_df.seed2,test_size=0.3,random_state=1)

In [176]:
#数据集
import xgboost as xgb
dtrain2=xgb.DMatrix(X_train2,y_train2)
dtest2=xgb.DMatrix(X_test2,y_test2)

In [177]:
#设置XGBoost模型
param2={
    #General Parameters
    'booster':'gbtree',#可选择gbtree和gblinear，默认为gbtree
    'silent':1,#取0时表示打印出运行时信息，取1时表示以缄默方式运行
    
    #Booster Parameters
    'eta':0.3,# 学习率，默认为0.3
    'max_depth':15,#树的最大深度，默认为6
    'min_child_weight':1,
    'gamma':0.1,
    'subsample':0.7,# 用于训练模型的子样本占整个样本集合的比例
    'colsample_bytree':1,# 在建立树时特征采样的比例
    
    # Task Parameters
    'objective':'binary:logistic',
    'eval_metric':'auc',# 评价标准
}

#训练次数
num_round=10
#评估结果
evallist2=[(dtest2,'eval'),(dtrain2,'train')]
#训练
bst2 = xgb.train(param2,dtrain2,num_round,evallist2)

[0]	eval-auc:0.751909	train-auc:0.746574
[1]	eval-auc:0.839505	train-auc:0.839789
[2]	eval-auc:0.837147	train-auc:0.842943
[3]	eval-auc:0.843092	train-auc:0.851402
[4]	eval-auc:0.843116	train-auc:0.852896
[5]	eval-auc:0.84226	train-auc:0.857545
[6]	eval-auc:0.847019	train-auc:0.865203
[7]	eval-auc:0.850877	train-auc:0.872291
[8]	eval-auc:0.851408	train-auc:0.883136
[9]	eval-auc:0.851475	train-auc:0.89003


In [178]:
seed2_full=seed_one.merge(df,on='id',how='left')
seed2_full.fillna(0)
seed2_full

,id,seed1,tag_128,tag_129,tag_130,tag_131,tag_132,tag_134,tag_135,tag_136,...,aoi_火车站,aoi_研究院,aoi_科技馆,aoi_美术馆,aoi_航站楼,aoi_购物中心,aoi_酒店,aoi_长途汽车站,aoi_陵园,aoi_高尔夫球场
0,109845,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121203,1,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,134044,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109445,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120921,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,8490,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11360,16673,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11361,3347,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11362,14462,1,0.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
y1=seed1_df.seed1
x1=seed1_df.drop(['id','seed1'],axis=1)

In [180]:
#测试全部数据集的seed1
integral_data1=xgb.DMatrix(x1,y1)
y_1=bst1.predict(integral_data1)
#查看预测结果
output1=pd.DataFrame(columns=['id','pro']) 
output1['id']=seed1_df['id']
output1['pro']=y_1
output1.set_index('id')
#_df.drop(['seed1','id']

,pro
id,
3.0,0.476041
4.0,0.478433
6.0,0.461897
7.0,0.464212
8.0,0.495948
...,...
141036.0,0.456995
141039.0,0.456226
141040.0,0.452257


In [181]:
output1.to_csv('seed1_results')

In [182]:
y2=seed2_df.seed2
x2=seed2_df.drop(['id','seed2'],axis=1)

In [183]:
#测试全部数据集的seed2
integral_data2=xgb.DMatrix(x2,y2)
y_2=bst2.predict(integral_data2)
#查看预测结果
output2=pd.DataFrame(columns=['id','pro']) 
output2['id']=seed2_df['id']
#output2['output']=output2['output'].astype(np.float)
#output2['output']=output2['output'].astype(str)
output2['pro']=y_2
output2.set_index('id')
output2

,id,pro
0,3,0.022414
1,4,0.022414
2,6,0.022414
3,7,0.024163
4,8,0.026286
...,...,...
53306,141036,0.022414
53307,141039,0.022414
53308,141040,0.044383
53309,141047,0.033369


In [184]:
output2.to_csv('seed2_results')